In [1]:
import os,glob
from wbtool import file_misc as fm
from wbtool.show import wshow,wplot
import numpy as np
from importlib import reload

cell_mask_dir = '/home/wb/samba_dir/cells_brightfield/process/lacss_point_gfp_d_e60000/' 
nuclear_mask_dir = '/home/wb/samba_dir/cells_brightfield/process/e6000_segs_cyto/' 
output_dir = '/home/wb/samba_dir/cells_brightfield/process/lacss_point_gfp_d_e60000_processed/' 

import skimage.morphology as skmorph
# from scipy import ndimage
# from skimage import measure
from onevision import morph_data, morphology
import aux
import proc_data


/home/wb/apps/miniconda3/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# xa = fm.imread(os.path.join(data_dir, 'D1ROI1_02_3_1_GFP_001_ne4c.png'))
# print(xa.shape)
# print(xa.min(), xa.max())
# print(xa.dtype)

x1 = xa==23 #hole
x2 = xa==28 #side
x3 = xa==34 #split
x4 = xa==53 #half
x5 = xa==74 #rough
x6 = xa==80 #half

# wshow(xa==80) 

import skimage.morphology as skmorph

a = skmorph.convex_hull_image(x1)

In [ ]:
# batch process
# reload(aux)

flist = glob.glob(os.path.join(cell_mask_dir, '*.png'))
# flist = ['/home/wb/samba_dir/cells_brightfield/process/lacss_point_gfp_d_e60000/D4ROI13_02_3_1_GFP_001.png']
output_dir = '/home/wb/samba_dir/cells_brightfield/process/temp/' 
save_npy = True
is_doing_nuc = True

print('get {} files'.format(len(flist)))
for file in flist:
    fn = os.path.basename(file)
    print('doing', fn)

    xa = fm.imread(file)
    C = morph_data.IMbind(xa, 'map')
    
    # ids = morph_data.get_obj_id(xa)
    # mps = []
    # for id in ids:
    #     a = xa==id
    #     mps.append(a)
    # b = np.stack(mps)
    # C = morph_data.IMbind(b, 'masks', ids=ids)

    #=== remove small noise object, and noisy part of cell mask (each cell must be a connected shape.)
    sz = aux.cell_avg_size(C)
    thres = sz/20
    morphology.filter_object_size(C, thres, None, relative_size=False, in_place=True)
    C = morphology.isolate_object(C, keep_one=True)
    morphology.filter_object_size(C, thres, None, relative_size=False, in_place=True)

    #=== fill holes
    C = morphology.fill_holes(C)

    #=== remove duplicates 
    ids = C.get_ids()
    # - 1. get convec hull of object.
    c_fill = []
    for k in ids:
        c_fill.append(skmorph.convex_hull_image(C[k]))
    # - 2. determine overlap according to convex shell, but generate new based on original mask.
    c_fill = morph_data.IMbind(c_fill, 'masks', ids=ids)
    L = aux.determine_dup(c_fill)
    for it in L:
        union_mask = C[it[0]] | C[it[1]]
        del C[max(it[:2])]
        C[min(it[:2])] = union_mask

    #=== merge/split masks according to nuclear mask.
    if is_doing_nuc:
        # locate corresponding nuclear file.
        fn_nuc, mark = aux.find_correspond_dapi_file(fn, nuclear_mask_dir)
        try:
            if not os.path.isfile(fn_nuc):
                raise Exception('nuclear file not found')
            K = fm.imread(fn_nuc)
            K = morph_data.IMbind(K, 'map')
            
            # wshow(proc_data.colorize_mask(C))
            C = aux.process_according_nuclear_mask(C, K)
            # wshow(proc_data.colorize_mask(morph_data.imbind_to_map(A)))
            
            
        except Exception as e:
            print(fn, 'process error')
            print(str(e))
            if len(C)==0:
                print('no cell exist after processing')
                continue
    
    #=== save
    fn_out = fn+'_processed_GFP'
    if mark == 'ne4c':
        fn_out += 'ne4c'

    if save_npy:
        _, temp = morph_data.imbind_to_masks(C)
        np.save(os.path.join(output_dir, fn_out), temp)
    proc_data.save_prediction(morph_data.imbind_to_map(C), output_dir, fn_out+'.png')

    # d = morph_data.imbind_to_map(C)
    # proc_data.save_prediction(d, output_dir, fn)
    print('done')



In [ ]:
# glob.glob(os.path.join(nuclear_mask_dir, '[AB]*.png'))
os.path.isfile('/home/wb/samba_dir/cells_brightfield/process/e6000_segs_cyto/C6ROI6_02_2_1_DAPI_001_cp_masks.png ')

In [29]:
import skimage.segmentation as skseg
from scipy import ndimage
# help(ndimage.distance_transform_edt)
# help(skseg.watershed)
help(ndimage.center_of_mass)
# M = skseg.watershed(-distance, marker, mask=img)

Help on function center_of_mass in module scipy.ndimage._measurements:

center_of_mass(input, labels=None, index=None)
    Calculate the center of mass of the values of an array at labels.
    
    Parameters
    ----------
    input : ndarray
        Data from which to calculate center-of-mass. The masses can either
        be positive or negative.
    labels : ndarray, optional
        Labels for objects in `input`, as generated by `ndimage.label`.
        Only used with `index`. Dimensions must be the same as `input`.
    index : int or sequence of ints, optional
        Labels for which to calculate centers-of-mass. If not specified,
        the combined center of mass of all labels greater than zero
        will be calculated. Only used with `labels`.
    
    Returns
    -------
    center_of_mass : tuple, or list of tuples
        Coordinates of centers-of-mass.
    
    Examples
    --------
    >>> import numpy as np
    >>> a = np.array(([0,0,0,0],
    ...               [0,1,